
# Vorlesung *Angewandte Softwaretechnik*
## Nebenläufigkeit

## Exkurs: Lokale und globale Variablen

**Warnung:** Globale Variablen sind zu vermeiden.

In [ ]:
def foo():
    print(n)

n = 23
foo()

- **Erklärung:** Die Variable `n` wurde *außerhalb* einer Funktion deklariert. Wir sagen: Sie ist **global**.
    - `n` kann **sowohl** aus dem *globalen* Kontext (aus dem "Hauptprogramm") 
    - **als auch** aus Funktionen heraus aufgerufen werden.
- **Merke:** Variablen, die *außerhalb* von Funktionen deklariert werden, nennen wir **global**.

In [ ]:
def bar():
    n = 42
    print(n)

n = 23
bar()
print(n)

- **Erklärung:** Im obigen Programm existiert **sowohl** eine *lokale* Variable `n` (Wert: 42) **als auch** eine *globale* Variable `n` (Wert: 23).
- **Ursache:** Wir haben den Wert von `n` in `bar()` nicht nur benutzt, sondern wir haben `n` einen Wert **zugewiesen**.
- **Merke:** Immer wenn wir in einer *Funktion* einer Variable einen Wert *zuweisen*, wird eine **lokale** Variable angelegt.

In [ ]:
def bar():
    n = 42
    print('lokal: n = %s' % n)
    
n = 23
bar()
print('global: n = %s' % n)

- **Merke:** Wir können *globale* Variablen aus Funktionen zwar **lesen** aber -- im Normalfall -- **nicht verändern** (weil sonst eine *lokale* Variable angelegt wird).

In [ ]:
def foo():
    global n
    n = 42

n = 23
foo()
print(n)

- **Merke:** Mit dem Schlüsselwort `global` erlauben wir einer Funktion den **schreibenden** Zugriff auf eine *globale* Variable.

### Warum dieses Verhalten?

- **Frage:** Warum ist es sinnvoll, dass wir globale Variablen zwar ohne weiteres **lesen**, aber nur nach ausdrücklicher Erklärung **schreiben** dürfen?

- **Antwort:** Dadurch vermeidet Python **unerwartete Seiteneffekte**.
- **Beispiel:** Es könnte ja sein, dass Sie nur eine lokale Variable `n` verwenden möchten und gar nicht *wissen*, dass es auch eine globale Variable `n` gibt. In diesem Fall, sind Sie vermutlich froh, dass die Zeile `n = 42` im Normalfall nur eine lokale Variable anlegt, und nicht die globale Variable `n` überschreibt.

In [ ]:
def inc():
    global n
    n += 1

def dec():
    global n
    n -= 1

n = 4710
dec()
inc()
inc()
print(n)

In [ ]:
def erhöhe_n():
    global n
    n += 1

def erhöhe(wert):
    return wert + 1

n = 21

erhöhe_n()
n = erhöhe(n)

print(n)

**Frage:** Welche der beiden obigen Funktionen erzeugt eher Probleme, die Sie sehr schwer debuggen können?

### Seiteneffekte

Wenn eine Funktion den Wert einer globalen Variablen verändert, sagen wir: die Funktion hat einen **Seiteneffekt**.

#### Nachteile
- Beim Programmieren ist nicht ohne weiteres zu erkennen, ob eine Funktion einen Seiteneffekt hat: Seiteneffekte begünstigen daher **Programmierfehler**.
- Seiteneffekte machen Code schwerer verständlich. Fehler, die durch Seiteneffekte entstehen, sind **schwer zu debuggen**.
- Seiteneffekte machen Funktionen **schwieriger zu testen**.

**Merke:** Vermeiden Sie **Seiteneffekte**, wo immer es möglich ist.

**Merke:** Vermeiden Sie **globale Variablen**, wo immer es möglich ist.

## Nebenläufigkeit und Parallelität

Was wir meinen, wenn wir "gleichzeitig" sagen.

### Was bedeutet "Parallelität"?

- **Parallelität:** Zwei Befehle werden **physikalisch gleichzeitig** ausgeführt.
<img src='diag_parallel.svg' height=20%>

- **Zweck:** Beschleunigung von Berechnungen (Stichwort: *parallele Algorithmen*).
- **Realisierung** auf Hardware-Ebene auf modernen Mehr-Kern-Prozessoren.

### Was bedeutet "Nebenläufigkeit"?

- **Nebenläufigkeit:** Zwei **Vorgänge** werden **im gleichen Zeitraum** ausgeführt.
<img src='diag_concurrent.svg' hieght=20%>

- **Zweck:** Verschiedene Vorgänge sollen sich gegenseitig nicht blockieren. 
  - *Beispiel*: Eine Benutzeroberfläche soll während einer langwierigen Berechnung nicht blockieren. Die Berechnung soll *im Hintergrund* ablaufen.

- **Realisierung** ist rein auf Softwareebene möglich. (Verarbeitung schaltet zwischen verschiedenen Prozessen hin und her.)

### Threads vs. Prozesse

**Threads** und **Prozesse** sind zwei Formen, Nebenläufigkeit zu realisieren.

- **Prozesse**:
    - werden vom *Betriebssystem* verwaltet,
    - arbeiten im Allgemeinen auf *getrennten Ressourcen* (Kommunikation ist aber möglich).
    - *Beispiel:* Auf dem Handy laufen gleichzeitig *Spotify* und *Telegram*.

- **Threads**:
    - sind nebenläufige Abläufe **innerhalb** eines Programms,
    - arbeiten auf den **gleichen** Daten (viele neue **Fehlerquellen**).
    - *Beispiel:* Sie können WhatsApp-Nachrichten empfangen, während *gleichzeitig* ein Upload läuft.

### Zusammenfassung
- **Nebenläufigkeit** bedeutet: mehrere Vorgänge laufen im gleichen Zeitraum.
- **Parallelität** ist ein *Spezialfall der Nebenläufigkeit*.
- *Nebenläufigkeit* kann **mit und ohne** Parallelität realisiert werden.
- **Prozesse** werden vom Betriebssystem verwaltet und arbeiten auf *getrennten* Ressourcen.
- **Threads** existieren *innerhalb* eines Programms und arbeiten auf den *gleichen* Daten.

### Unser Thema: *Nebenläufigkeit* mit *Threads*
- Wie arbeite ich mit Threads?
- Welche Probleme können auftreten?
- Wie löse ich diese Probleme?

### Das Modul `threading`

In [1]:
from time import sleep

def slow(text):
    for char in text:
        sleep(0.5)
        print(char, end='')
    
slow('ASOTE')

ASOTE

In [2]:
from threading import Thread

#slow('foo')
t = Thread(target=slow, args=('foo',))
t.start()

foo

- **Frage:** Ist das nur eine komische Art, Funktionen aufzurufen?

In [3]:
t1 = Thread(target=slow, args=('foo',))
t2 = Thread(target=slow, args=('bar',))
t1.start()
t2.start()

fboaro

- **Frage:** Also laufen die beiden Threads parallel?
- **Antwort:** Nein, sie laufen **nebenläufig**.

**Merke:**
1. Beide Parameter sind **benannt**:
    - <code>t1 = Thread(<mark>target</mark>=slow, <mark>args</mark>=('foo',))</code>
1. Als `target` übergeben wir nur den **Namen** der Methode (ohne `()`):
    - <code>t1 = Thread(target=<mark>slow</mark>, args=('foo',))</code>
1. Die Parameter werden als **Tupel** übergeben:
    - <code>t1 = Thread(target=slow, args=<mark>('foo',)</mark>)</code>


In [4]:
param1 = 'foo'
param2 = ('foo')
param3 = ('foo', )
print(type(param1))
print(type(param2))
print(type(param3))     

<class 'str'>
<class 'str'>
<class 'tuple'>


**Merke:** Das zusätzliche **Komma** in `('foo',)` ist notwendig, um ein **1-elementiges Tupel** zu kennzeichnen.


In [5]:
t1 = Thread(target=slow, args=('foo',))
t2 = Thread(target=slow, args=('bar',))
t1.start()
t2.start()

print('Hier endet das Programm. Vielen Dank und gute Nacht!')

Hier endet das Programm. Vielen Dank und gute Nacht!
fboaro

- **Merke:** Ein Thread kann länger "leben" als das Hauptprogramm.

In [ ]:
t1 = Thread(target=slow, args=('foo',))
t2 = Thread(target=slow, args=('bar',))
t1.start()
t2.start()
t1.join()
t2.join()
print('\nHier endet das Programm. Vielen Dank und gute Nacht!')

- **Merke:** Mit `Thread.join()` warten wir, bis ein bestimmter Thread fertig ausgeführt wurde.

### Anwendungsbeispiel: Nebenläufige Webcalls

In [ ]:
import json
import requests
from pprint import pprint

url = 'https://restcountries.eu/rest/v2/name/Germany'
res = requests.get(url)
if (res.status_code == 200):
    print(res.content.decode()[:250])

In [ ]:
import json
import requests

def finde_hauptstadt(name):
    url = 'https://restcountries.eu/rest/v2/name/%s' % name
    res = requests.get(url)
    if (res.status_code == 200):
        land = json.loads(res.content.decode())
        return land[0]['capital']
    
print(finde_hauptstadt('Cook Islands'))    

#### Zeitmessung

In [ ]:
import time

jetzt = time.time()

print(jetzt)
print(type(jetzt))

In [ ]:
import time

tic = time.time()
h1 = finde_hauptstadt('Indonesia')
print(h1)
toc = time.time()

print("Zeit: %.3f Sekunden." % (toc-tic))

In [ ]:
import time

tic = time.time()
h1 = finde_hauptstadt('Morocco')
h2 = finde_hauptstadt('Egypt')
print(h1)
print(h2)
toc = time.time()

print("Zeit: %.3f Sekunden." % (toc-tic))

**Wir stellen fest:** Webcalls nacheinander abzusetzen ist ineffizient.

In [ ]:
from threading import Thread

tic = time.time()

t1 = Thread(target=finde_hauptstadt, args=('Morocco',))
t2 = Thread(target=finde_hauptstadt, args=('Egypt',))
t1.start()
t2.start()
t1.join()
t2.join()
toc = time.time()

print("Zeit: %.3f Sekunden." % (toc-tic))

- **Wir stellen fest:** *Nebenläufige* Webcalls sind wesentlich effizienter.
- **Aber:** Wie kommen wir jetzt an das Ergebnis der Webcalls?

**Merke:** Wenn wir eine Funktion in einem eigenen Thread aufrufen, haben wir **keine** Möglichkeit, den Rückgabewert der Funktion zu nutzen.

In [ ]:
def map_hauptstadt(land_name, ziel_dict):
    hauptstadt = finde_hauptstadt(land_name)
    ziel_dict[land_name] = hauptstadt
    
capitals = {}
map_hauptstadt('Argentina', capitals)
print(capitals)

**Lösung:** Wir hinterlegen die Ergebnisse in einem Dictionary.

In [ ]:
tic = time.time()

capitals = {}
t1 = Thread(target=map_hauptstadt, args=('Morocco', capitals))
t2 = Thread(target=map_hauptstadt, args=('Egypt', capitals))
t1.start()
t2.start()
t1.join()
t2.join()
print(capitals)

toc = time.time()

print("Zeit: %.3f Sekunden." % (toc-tic))

In [ ]:
tic = time.time()
laender = ['Belgien', 'Chile', 'Benin', 'Laos']
capitals = {}
threads = []
for land in laender:
    thread = Thread(target=map_hauptstadt, args=(land, capitals))
    thread.start()
    threads.append(thread) 
for thread in threads:  
    thread.join()
print(capitals)
toc = time.time()
print("Zeit: %.3f Sekunden." % (toc-tic))

## Race Conditions

- **Wir stellen fest:** Mit Threads können wir mehrere Dinge **"gleichzeitig"** zu tun.
- **Frage:** Hat diese Technik auch Nachteile?

- **Antwort:** Leider **ja**.
- **Vor allem:** viele neue Fehlerquellen. Noch dazu solche, die sich mit dem *Debugger* schlecht finden lassen.

### Beispiel: Bankkonten

In [ ]:
from pprint import pprint
import time

konto = {'nummer': 'A123',
         'saldo': 1000,
         'limit': -1000}

pprint(konto)

- `'nummer'` = Kontonummer
- `'saldo'` = der aktuelle Kontostand
- `'limit'` = das Kreditlimit (nach jeder Buchung muss gelten: `saldo` $\geq$ `limit`)


In [ ]:
def verbuche(betrag, konto):
    saldo_neu = konto['saldo'] + betrag
    if saldo_neu >= konto['limit']:
        konto['saldo'] = saldo_neu

konto = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000}
verbuche(-2500, konto)
print(konto)

- Die Funktion `verbuche(<betrag>, <konto>)` führt eine Kontobuchung mit Prüfung des Kreditlimits durch.

In [ ]:
from numpy import random

def verbuche(betrag, konto):
    saldo_neu = konto['saldo'] + betrag
    if saldo_neu >= konto['limit']:
        time.sleep(0.01 * random.rand()) # künstliche Verzögerung
        konto['saldo'] = saldo_neu

- Wir bauen eine **künstliche Verzögerung** ein, weil das Problem, das wir zeigen wollen, sonst sehr selten auftritt.

- Auch in der **Wirklichkeit** kommen beim Programmablauf **Verzögerungen** vor. Diese sind aber weniger gut kontrollierbar.

In [ ]:
konto = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000}
verbuche(-600, konto)
print(konto)
verbuche(-600, konto)
print(konto)
verbuche(-600, konto)
print(konto)
verbuche(-600, konto)
print(konto)

In [ ]:
from threading import Thread

konto = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000}
t1 = Thread(target=verbuche, args=(500, konto))
t2 = Thread(target=verbuche, args=(-500, konto))
t1.start()
t2.start()
t1.join()
t2.join()

print(konto)    

**Bebachtung:** Wenn wir mit mehreren Threads **gleichzeitig** auf das Konto zugreifen, ist der Saldo (zufällig) `500` oder `1500`. **Warum?**

In [ ]:
def verbuche(betrag, konto):
    nummer = konto['nummer']
    saldo = konto['saldo']
    limit = konto['limit']
    saldo_neu = saldo + betrag
    if saldo_neu >= limit:
        time.sleep(0.01 * random.rand()) # künstliche Verzögerung
        konto['saldo'] = saldo_neu
        print('[%s] %s + (%s) = %s' % (nummer, saldo, betrag, saldo_neu))
    else:
        print('[%s] Buchung abgelehnt: %s' % (nummer, betrag))

- Wir versehen die Funktion mit ausführlicher **Ausgabe**.

In [ ]:
from threading import Thread

konto = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000}
t1 = Thread(target=verbuche, args=(500, konto))
t2 = Thread(target=verbuche, args=(-500, konto))
t1.start()
t2.start()
t1.join()
t2.join()

print(konto)

### Möglicher Ablauf 1

| Thread 1 | Thread 2 | Konto |
| --- | --- | --- |
| `betrag = 500`| `betrag = -500`| <code>{'nummer': 'A123', <mark>'saldo': 1000</mark>, 'limit': -1000}</code> |
| `saldo = 1000` | |
| `saldo_neu = 1500` | |
| &nbsp; | `saldo = 1000` | 
| &nbsp; | `saldo_neu = 500` | 
| &nbsp; | `konto['saldo'] = 500` | 
| `konto['saldo'] = 1500` | |
| | | <code>{'nummer': 'A123', <mark>'saldo': 1500</mark>, 'limit': -1000}</code> |


### Möglicher Ablauf 2

| Thread 1 | Thread 2 | Konto |
| --- | --- | --- |
| `betrag = 500`| `betrag = -500`| <code>{'nummer': 'A123', <mark>'saldo': 1000</mark>, 'limit': -1000}</code> |
| `saldo = 1000` | |
| `saldo_neu = 1500` | |
| &nbsp; | `saldo = 1000` | 
| &nbsp; | `saldo_neu = 500` | 
| `konto['saldo'] = 1500` | |
| &nbsp; | `konto['saldo'] = 500` | 
| | | <code>{'nummer': 'A123', <mark>'saldo': 500</mark>, 'limit': -1000}</code> |


### Möglicher Ablauf 3

| Thread 1 | Thread 2 | Konto |
| --- | --- | --- |
| `betrag = 500`| `betrag = -500`| <code>{'nummer': 'A123', <mark>'saldo': 1000</mark>, 'limit': -1000}</code> |
| `saldo = 1000` | |
| `saldo_neu = 1500` | |
| `konto['saldo'] = 1500` | |
| &nbsp; | `saldo = 1500` | 
| &nbsp; | `saldo_neu = 1000` | 
| &nbsp; | `konto['saldo'] = 1000` | 
| | | <code>{'nummer': 'A123', <mark>'saldo': 1000</mark>, 'limit': -1000}</code> |


### Race Condition *("Wettbewerbs-Situation")*

- *In der Elektronik*: Jede Situation, in der das Ergebnis einer Berechnung davon abhängt, in welcher Reihenfolge Signale eintreffen.
- **In der Informatik:** Eine Situation, in der die **nebenläufige Ausführung** zu einem **unerlaubten Zustand** des Programms führt.

**Merke:** Wenn ein Programm bei **nebenläufiger** Ausführung in einen unerlaubten Zustand gerät, der bei **sequentieller** (=nicht nebenläufiger) Ausführung nicht erreichbar gewesen wäre, haben wir eine **Race Condition**.

### Geteilte Ressourcen *(shared resources)*
- Das Dictionary `konto` ist ein Beispiel für eine **geteilte Ressource**.
  - **Resource** = ein **Gerät** oder eine **Information**, worauf wir per Software zugreifen können
  - **geteilt** = von mehreren Threads (oder Prozessen) verwendet
  

- Eine **Race Condition** tritt auf, wenn
  - mehrere Threads (oder Prozesse) **gleichzeitig** auf eine geteilte Resource zugreifen
  - und mindestens einer davon die Resource **verändert**.
  
**Merke:** Wird eine geteilte Ressource von allen Threads nur *gelesen* haben wir kein Problem.

<center>
<div style="position:relative">
    <img style="width:70%" src='bread.jpg'>
    <div style="position:absolute;bottom:50px;left:50%">
        <h2 style="background-color:white">&nbsp;Das Bäcker-Problem&nbsp;</h2>
        <p style="background-color:white">&nbsp;Wie wir Race Conditions verhindern können&nbsp;</p>
    </div>
</div>
</center>

### Das  Bäcker-Problem
| Person A | Person B |
| --- | --- |
| Es ist kein Brot da |  |
| Zum Bäcker gehen |  |
| &nbsp; | Es ist kein Brot da | 
| &nbsp; | Zum Bäcker gehen | 
| &nbsp; | Brot kaufen | 
| Brot kaufen |  |

**Ergebnis:** Wir haben zwei Laib Brot.

<h3>Die Lösung: Das Brot-Token</h3>
<center>
<img src='signboard-brot.png' width=40%>
</center>

- **Lösung:** Ich gehe nur zum Bäcker, wenn das Brot-Token da ist. Wenn es da ist, nehme ich es mit.
- **Erkenntnis:** Wir brauchen eine Art **Token** um sicherzustellen, dass nur **ein Thread gleichzeitig** eine kritische Resource (z.B. `konto`) bearbeitet.

In [ ]:
from threading import Lock

konto_lock = Lock() # ein Token für Kontobuchungen

- **Merke:** Wir importieren die Klasse `Lock` aus `threading`.
- **Merke:** Mit der Anweisung `Lock()` erstellen wir ein Lock-Token. (Dieses sollten wir in einer Variable hinterlegen z.B. `lock`.)

In [ ]:
from threading import Lock
from numpy import random
from time import sleep

konto_lock = Lock()

def verbuche(betrag, konto):
    with konto_lock: # Blockiere mit globalem Lock-Token
        saldo_neu = konto['saldo'] + betrag
        if saldo_neu >= konto['limit']:
            sleep(0.01 * random.rand()) # künstliche Verzögerung
            konto['saldo'] = saldo_neu

- **Merke:** `with <lock>:` bewirkt, dass der eingerückte Block:
    1. nur ausgeführt wird, wenn das Lock-Token **verfügbar** ist und
    2. während der Ausführung das Lock-Token **blockiert** (d.h. es ist für andere Threads *nicht verfügbar*.)

### Fachbegriff: Synchronisierung

Das Schützen eines Code-Abschnitts vor der Ausführung durch mehrere Threads gleichzeitig nennen wir **synchronisieren**.

Ein so geschützter Code-Abschnitt heißt **synchronisiert**.

In [ ]:
from threading import Thread

konto = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000}
t1 = Thread(target=verbuche, args=(500, konto))
t2 = Thread(target=verbuche, args=(-500, konto))
t1.start()
t2.start()
t1.join()
t2.join()

print(konto)    

- **Beobachtung:** Das Synchronisieren verhindert das Auftreten der *Race Condition*.

## Kritische Abschnitte

Welche Abschnitte sollten wir synchronisieren?

### Kritischer Abschnitt *(critical section)*
- Einen Code-Abschnitt, der *synchronisiert* werden muss um eine *Race Condition* zu verhindern bezeichnen wir als **kritischen Abschnitt**.
- Die *Herausforderung* besteht darin, kritische Abschnitte richtig zu **erkennen**.
- Auf den folgenden Folien zeigen wir **typische Beispiele** für kritische Abschnitte.

### Beispiel 1: *Check-then-act*-Situationen

Geteilte Resource: `liste`

`if wert in liste:
    liste.remove(wert)`

<table>
    <tr>
        <th style="text-align:center">Thread A</th>
        <th style="text-align:center">Thread B</th>
        <th style="text-align:center">Programmzustand</th>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td><em>liste == [1, 2, 4]</em></td>
    </tr>
    <!--tr>
        <td style="text-align:center"><em>(wert == 2)</em></td>
        <td style="text-align:center"><em>(wert == 2)</em></td>
        <td></td>        
    </tr-->
    <tr>
        <td style="text-align:left"><code>if 2 in liste:</code></td>
        <td></td>
        <td></td>
    </tr>    
    <tr>
        <td></td>
        <td style="text-align:left"><code>if 2 in liste:</code></td>
        <td></td>
    </tr>    
    <tr>
        <td style="text-align:left"><code>    liste.remove(2)</code></td>
        <td></td>
        <td><em>liste == [1, 4]</em></td>
    </tr>    
    <tr>
        <td></td>
        <td style="text-align:left"><code>    liste.remove(2)</code></td>
        <td><em>ValueError</em></td>
    </tr>    
</table>

**Merke:** *Check-then-act*-Abschnitte (auf geteilten Ressourcen) sind **kritisch**.

### Beispiel 1: *Check-then-act*-Situation, synchronisiert

<code>lock_liste = Lock()</code>
<p>...</p>
<code>with lock_liste:
    if wert in liste:
        liste.remove(wert)</code>
        
**Merke:** *Check-then-act*-Abschnitte müssen stets *als ganzes* synchronisiert werden.

### Beispiel 2: *Read-modify-write*-Situationen

Geteilte Ressource: `zaehler`

<code>global zaehler

alter_wert = zaehler
neuer_wert = alter_wert + 1
zaehler = neuer_wert</code>

<table>
    <tr>
        <th style="text-align:center">Thread A</th>
        <th style="text-align:center">Thread B</th>
        <th style="text-align:center">Programmzustand</th>
    </tr>
    <!--tr>
        <td></td>
        <td></td>
        <td><em>zaehler == 0</em></td>
    </tr-->
    <!--tr>
        <td style="text-align:center"><em>(wert == 2)</em></td>
        <td style="text-align:center"><em>(wert == 2)</em></td>
        <td></td>        
    </tr-->
    <tr>
        <td style="text-align:left"><code>alter_wert = zaehler</code> <em>(0)</em></td>
        <td></td>
        <td><em>zaehler == 0</em></td>
    </tr>    
    <tr>
        <td style="text-align:left"><code>neuer_wert = alter_wert + 1</code> <em>(1)</em></td>
        <td></td>
        <td></td>
    </tr>    
    <tr>
        <td></td>
        <td style="text-align:left"><code>alter_wert = zaehler</code> <em>(0)</em></td>
        <td></td>
    </tr>    
    <tr>
        <td></td>
        <td style="text-align:left"><code>neuer_wert = alter_wert + 1</code> <em>(1)</em></td>
        <td></td>
    </tr>  
    <tr>
        <td></td>
        <td style="text-align:left"><code>zaehler = neuer_wert</code></td>
        <td><em>zaehler == 1</em></td>
    </tr>  
    <tr>
        <td style="text-align:left"><code>zaehler = neuer_wert</code></td>
        <td></td>
        <td><em>zaehler == 1</em></td>
    </tr>  
</table>

**Merke:** *Read-modify-write*-Abschnitte (auf geteilten Resourcen) sind **kritisch**.

### Beispiel 2: *Read-modify-write*-Situation, synchronisiert

<code>zaehler_lock = Lock()</code>
<p>...</p>
<code>global zaehler
with zaehler_lock:
    alter_wert = zaehler
    neuer_wert = alter_wert + 1
    zaehler = neuer_wert</code>
    
**Merke:** Auch *Read-modify-write*-Abschnitte müssen stets *als ganzes* synchronisiert werden.    

### Beispiel 3: Modifizierende Zuweisungen

Geteilte Ressource: `zaehler`

`global zaehler
zaehler += 1`



- **Frage:** Könnte die Anweisung `zaehler += 1` ein kritischer Abschnitt sein?

- **Antwort:** *Ja*.
- **Begründung:** Intern wird `zaehler += 1` wie das *Read-modify-write*-Beispiel auf der vorigen Folie ausgeführt.

### Beispiel 3: Modifizierende Zuweisung, synchronisiert

<code>zaehler_lock = Lock()</code>
<p>...</p>
<code>global zaehler
with zaehler_lock:
    zaehler += 1</code>

### Beispiel 4: Einfache Zuweisung

Geteilte Resource: `budget` 

`global budget
budget = 3`

**Frage:** Kann eine einfache Zuweisung kritisch sein?

### Einfache Zuweisung + *Check-then-act*

Geteilte Ressource: `budget`

<table>
    <tr>
        <th style="text-align:center">Thread A</th>
        <th style="text-align:center">Thread B</th>
    </tr>
    <tr>
        <td style="text-align:left;vertical-align:top"><code>budget = 3</code></td>
        <td style="text-align:left;vertical-align:top"><code>if budget >= 5:
    budget -= 5</code></td>
    </tr>
</table>

        
<table>
    <tr>
        <th style="text-align:center">Thread A</th>
        <th style="text-align:center">Thread B</th>
        <th style="text-align:center">Programmzustand</th>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td><em>budget == 12</em></td>
    </tr>
    <!--tr>
        <td style="text-align:center"><em>(wert == 2)</em></td>
        <td style="text-align:center"><em>(wert == 2)</em></td>
        <td></td>        
    </tr-->
    <tr>
        <td></td>
        <td style="text-align:left"><code>if budget > 5:</code> <em>(True)</em></td>
        <td></td>
    </tr>    
    <tr>
        <td style="text-align:left"><code>budget = 3</code></td>
        <td></td>
        <td><em>budget == 3</em></td>
    </tr>    
    <tr>
        <td></td>
        <td style="text-align:left"><code>budget -= 5</code></td>
        <td><em>budget == -2</em></td>
    </tr>    
</table>

- **Merke:** Auch eine **einfache Zuweisung** kann (im Zusammenspiel mit anderen Codeabschnitten) eine *Race Condition* auslösen -- ist also ein **kritischer Abschnitt**.

### Einfache Zuweisung + *Check-then-act*, synchronisiert

<code>bugdet_lock = Lock()</code>
<p>...</p>
<code>with budget_lock:
    budget = 3</code>
<p>...</p>
<code>with budget_lock:
    if budget >= 5:
        budget -= 5</code>    

### Beispiel 5: Leseoperationen

Geteilte Resource: `anschrift` *(Dictionary)*

<code>
    print(anschrift['straße'])
    print(anschrift['nummer'])</code>

**Frage:** Kann eine einfache Lese-Operation (wie `print(anschrift['straße'])`) kritisch sein?

### Beispiel: Zusammengehörende Lese- und Schreiboperationen


<table>
    <tr>
        <th style="text-align:center">Thread A</th>
        <th style="text-align:center">Thread B</th>
        <th style="text-align:center">Programmzustand</th>
    </tr>
    <tr>
        <td></td>
        <td></td>
        <td style="text-align:left;vertical-align:top"><em>anschrift == {'straße': 'Prittwitzstr.', 'nummer': 10}</em>
    </tr>    
    <tr>
        <td style="text-align:left;vertical-align:top"><code>print(anschrift['straße'])</code> <em>(Prittwitzstr)</em></td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td></td>
        <td style="text-align:left;vertical-align:top"><code>anschrift['straße'] = 'Albert-Einstein-Allee'</code></td>
        <td style="text-align:left;vertical-align:top"><em>anschrift == {'straße': 'Albert-Einstein-Allee', 'nummer': 10}</em>
    </tr>
    <tr>
        <td></td>
        <td style="text-align:left;vertical-align:top"><code>anschrift['nummer'] = 55</code></td>
        <td style="text-align:left;vertical-align:top"><em>anschrift == {'straße': 'Albert-Einstein-Allee', 'nummer': 55}</em>
    </tr>
    <tr>
        <td style="text-align:left;vertical-align:top"><code>print(anschrift['nummer'])</code> <em>(55)</em></td>
        <td></td>
        <td></td>
    </tr>    
</table>

- **Ausgabe:** `Prittwitzstraße 55`
- **Merke:** Unsynchronisierte Leseoperationen können unerwartet einen **ungültigen Zwischenzustand** lesen.

### Zusammengehörende Lese- und Schreiboperationen, synchronisiert

<code>anschrift_lock = Lock()</code>
<p>...</p>
<code>with anschrift_lock:
    print(anschrift['straße'])
    print(anschrift['nummer'])</code>
<p>...</p>
<code>with anschrift_lock:
    anschrift['straße'] = 'Albert-Einstein-Allee'
    anschrift['nummer'] = 55</code>    

### Kann man zu viel synchronisieren?

**Frage:** Sollte man im Zweifel so viel wie möglich synchronisieren?

**Antwort:** Nein, auf keinen Fall.

- Wenn Sie zu viel Code synchronisieren, **blockieren** sich Threads gegenseitig.
- Der wesentlich **Vorteil** der Nebenläufigkeit -- Vorgänge, die einander nicht blockieren -- geht **verloren**.
    - Fachbegriff: Threads werden **"ausgehungert"**.

**Merke:** Synchronisieren Sie *so viel wie nötig* (kritische Abschnitte) und *so wenig wie möglich*.

### Genügt ein Lock-Token?

**Frage:** Kann ich alle kritischen Bereiche mit einem einzigen Lock-Token synchronisieren?

**Antwort:** Nein, damit würden Threads einander unnötig blockieren.

- *Race Conditions* treten nur auf, wenn Threads **gleichzeitig** auf die **gleiche Ressource** zugreifen.
- Threads die auf unterschiedliche Resourcen zugreifen, sollten einander **nicht blockieren**. (Gefahr des "*Aushungerns*".)

**Merke:** Erstellen Sie für jede geteilte Resource ein **eigenes** *Lock-Token*.

In [ ]:
from threading import Lock

n = 0
m = 0
lock_n = Lock()
lock_m = Lock()

def inc_n():
    global n
    with lock_n:
        n += 1
        
def inc_m():
    global m
    with lock_m:
        m += 1        

### Zusammenfassung: Kritische Abschnitte

- Ein Codeabschnitt, der (ohne Synchronisation) zu einer *Race Condition* führen kann, heißt **kritischer Abschnitt**.
- Ein *kritischer Abschnitt* besteht immer wenn auf eine **geteilte Ressource** zugegriffen wird.
- **Kritische Abschnitte** sind immer *so klein wie möglich* zu wählen, um das Blockieren und Aushungern von Threads zu vermeiden.
- In manchen Situationen **muss** ein kritischer Abschnitt aber **mehrere Zeilen** umfassen
    - z.B. *check-then-act*, *read-modify-write*, Zugriff auf *zusammengehörende Werte*.
- Um unnötiges Blockieren zu vermeiden, sollte für jede geteilte Ressource ein **eigenes Lock-Token** verwendet werden.

### Zeitmessung

In [ ]:
konto = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000}
tic = time.time()
t1 = Thread(target=verbuche, args=(500, konto))
t2 = Thread(target=verbuche, args=(-500, konto))
t1.start()
t2.start()
t1.join()
t2.join()
toc = time.time()
print('Endsaldo: %s' % konto['saldo'])    
print("Zeit: %.3f Sekunden." % (toc-tic))

In [ ]:
kontoA = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000}
kontoB = {'nummer': 'B042', 'saldo': 4711, 'limit': 0}
tic = time.time()
t1 = Thread(target=verbuche, args=(500, kontoA))
t2 = Thread(target=verbuche, args=(-500, kontoB))
t1.start()
t2.start()
t1.join()
t2.join()
toc = time.time()
print('Endsaldo: %s' % konto['saldo'])    
print("Zeit: %.3f Sekunden." % (toc-tic))

- **Beobachtung:** Auch Buchungen auf **unterschiedliche** Konten werden jetzt **nacheinander** ausgeführt.
- **Erklärung:** Wir haben nur **ein** globales Lock-Token.

- **Problem:** Jetzt blockiert das Lock-Token Buchungen auf **allen** Konten, nicht nur auf eines.
- **Frage:** Was fehlt noch?

- **Antwort:** Wir brauchen **für jedes Konto** ein Lock-Token!

In [ ]:
from pprint import pprint
import time
import random

konto = {'nummer': 'A123',
         'saldo': 1000,
         'limit': -1000,
         'lock': Lock()}

print(konto)

**Neue Datenstruktur** für Bankkonten: ein *Dictionary*
- `'nummer'` = Kontonummer
- `'saldo'` = der aktuelle Kontostand
- `'limit'` = das Kreditlimit (nach jeder Buchung muss der Kontostand $\geq$ dem `limit` sein)
- `'lock'` = das Lock-Token für das Konto


In [ ]:
def verbuche(betrag, konto):
    konto_lock = konto['lock'] # Hole Lock-Token aus Dictionary
    with konto_lock: # Blockiere mit globalem Lock-Token
        saldo_neu = konto['saldo'] + betrag
        if saldo_neu >= konto['limit']:
            time.sleep(0.01 * random.rand()) # künstliche Verzögerung
            konto['saldo'] = saldo_neu

In [ ]:
konto = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000, 'lock': Lock()}
tic = time.time()
t1 = Thread(target=verbuche, args=(500, konto))
t2 = Thread(target=verbuche, args=(-500, konto))
t1.start()
t2.start()
t1.join()
t2.join()
toc = time.time()
print('Endsaldo: %s' % konto['saldo'])    
print("Zeit: %.3f Sekunden." % (toc-tic))

In [ ]:
kontoA = {'nummer': 'A123', 'saldo': 1000, 'limit': -1000, 'lock': Lock()}
kontoB = {'nummer': 'B042', 'saldo': 4711, 'limit': 0, 'lock': Lock()}
tic = time.time()
t1 = Thread(target=verbuche, args=(500, kontoA))
t2 = Thread(target=verbuche, args=(-500, kontoB))
t1.start()
t2.start()
t1.join()
t2.join()
toc = time.time()
print('Endsaldo: %s' % konto['saldo'])    
print("Zeit: %.3f Sekunden." % (toc-tic))

**Beobachtung:** Jetzt werden Buchungen auf **verschiedenen** Konten **nebenläufig** ausgeführt. Nur Buchungen auf das **gleiche** Konto werden **nacheinander** ausgeführt.

## Deadlocks

Neue Fehlerquellen durch Lock Tokens

### Hinweis

Das folgende Beispiel ist unrealistisch stark vereinfacht. Es steht aber stellvertretend für viele reale, aber weniger übersichtliche Probleme

In [ ]:
def verleihe():
    global bestand
    global verliehen
    
    bestand_neu = bestand - 1
    verliehen_neu = verliehen + 1
    if bestand_neu >= 1:
        bestand = bestand_neu
        verliehen = verliehen_neu

In [ ]:
def nimmzurück():
    global bestand
    global verliehen
    
    bestand_neu = bestand + 1
    verliehen_neu = verliehen - 1
    if verliehen_neu >= 0:
        bestand = bestand_neu   
        verliehen = verliehen_neu        

In [ ]:
bestand = 2
verliehen = 8

verleihe()
verleihe()
nimmzurück()

print('%s/%s verliehen' % (verliehen, verliehen + bestand))    

### Bibliothekssystem mit Mehrbenutzerbetrieb

**Anforderung:** Wir möchten nun die Nutzung des Bibliothekssystems durch **mehrere Benutzer** simulieren.

**Ansatz:** Wir simulieren den Mehrbenutzerbetrieb durch zwei **Threads**:
1. ein Thread für den Ausleihe-Schalter und
2. ein Thread für den Rücknahme-Schalter.
    
Außerdem simulieren wir den Dauerbetrieb durch jeweils eine **Schleife**.   

**Frage:** Was müssen wir dabei beachten?

**Antwort:** Beide Funktionen enthalten **kritische Bereiche**. (Die kritischen Werte sind die Variablen `verfügbar` und `verliehen`.)

**Maßnahme:** Wir müssen mit *Lock Tokens* arbeiten, um Race Conditions zu verhindern.

In [ ]:
from time import sleep

def verleihe():
    global bestand
    global verliehen
    
    with(lock_bestand):
        with(lock_verliehen):    
            bestand_neu = bestand - 1
            verliehen_neu = verliehen + 1
            if bestand_neu >= 1:
                bestand = bestand_neu
                verliehen = verliehen_neu                

def verleihe_loop():
    for _ in range(1000):
        verleihe()
        print('%s/%s verliehen' % (verliehen, verliehen + bestand))

In [ ]:
def nimmzurück():
    global bestand
    global verliehen

    with(lock_verliehen):
        sleep(10 ** -6) # erhöhe die Wahrscheinlichkeit des Auftretens        
        with(lock_bestand):
            bestand_neu = bestand + 1
            verliehen_neu = verliehen - 1     
            if verliehen_neu >= 0:
                bestand = bestand_neu   
                verliehen = verliehen_neu
                
def nimmzurück_loop():
    for _ in range(1000):
        nimmzurück()  
        print('%s/%s verliehen' % (verliehen, verliehen + bestand)) 

In [ ]:
from threading import Thread, Lock

bestand = 10
verliehen = 0
lock_bestand = Lock()
lock_verliehen = Lock()

tV = Thread(target=verleihe_loop)
tN = Thread(target=nimmzurück_loop)

print("Starte Threads....")
tV.start()
tN.start()

- **Beobachtung:** Das Programm geht nicht mehr weiter (=> **Deadlock**).
- **Frage:** Warum?

### Günstiger Fall:
| verleihe_thread | zurück_thread |
| --- | --- |
| Warte auf `lock_bestand` |  |
| Sperre `lock_bestand` |  |
| Warte auf `lock_verliehen` |  |
| Sperre `lock_verliehen` |  |
| &nbsp; | Warte auf `lock_verliehen` |
| Modifiziere `bestand` und `verliehen` | |
| Gib `lock_verliehen` auf |  |
| | Sperre `lock_verliehen` |
| | Warte auf `lock_bestand` |
| Gib `lock_bestand` auf |  |
| | Sperre `lock_bestand` | 
| ... | ... |

### Ungünstiger Fall:
| verleihe_thread | zurück_thread |
| --- | --- |
| Warte auf `lock_bestand` |  |
| Sperre `lock_bestand` |  |
| &nbsp; | Warte auf `lock_verliehen` |  
| &nbsp; | Sperre `lock_verliehen` |  
| &nbsp; | Warte auf `lock_bestand` |  
| Warte auf `lock_verliehen` |  |

**=> Deadlock**

### Wann treten Deadlocks auf?
- Wenn **mehrere Threads** ...
- ... **mehrere Locks** benutzen ...
- ... und diese in **unterschiedlicher Reihenfolge** anfordern.

### Lösung:
- In Funktion `rechneA` steht: `with lock1: ... with lock2:` ...
- ... also schreiben wir in `rechneB` ebenfalls: `with lock1: ... with lock2:`
- (in dieser **Reihenfolge**).

In [ ]:
from time import sleep

def nimmzurück():
    global bestand
    global verliehen

    with(lock_bestand):          # Gleiche Reihenfolge ...
        sleep(10 ** -6)
        with(lock_verliehen):      # ... wie in verleihe()
            bestand_neu = bestand + 1
            verliehen_neu = verliehen - 1     
            if verliehen_neu >= 0:
                bestand = bestand_neu   
                verliehen = verliehen_neu
                print('%s/%s verliehen' % (verliehen, verliehen + bestand)) 

In [ ]:
bestand = 10
verliehen = 0
lock_verliehen = Lock()
lock_bestand = Lock()

tV = Thread(target=verleihe_loop)
tN = Thread(target=nimmzurück_loop)

print("Starte Threads....")
tV.start()
tN.start()

... läuft **endlos**.

### Deadlocks verhindern

**Merke:** Wenn wir mehrere Lock-Tokens verwenden, müssen wir darauf achten, dass die Tokens an **allen Stellen** immer in der **gleichen Reihenfolge** reserviert werden!